# Merge e Análise de Tabelas do CENIPA

Esse notebook contém um estudo exploratório da base de dados do CENIPA que está disponível publicamente [neste link](https://dados.gov.br/dataset/ocorrencias-aeronauticas-da-aviacao-civil-brasileira).

## Importando bibliotecas

In [ ]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 122 kB 8.2 MB/s 


In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
pd.set_option('display.max_columns',100)
#pd.set_option('display.max_rows', 10000)

## Extração dos Dados

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/aulas-bc26-dados-gbs/cenipa/ocorrencia.csv', sep=';', encoding='ISO-8859-1', parse_dates=['ocorrencia_dia'], dayfirst=True)
df1 = pd.read_csv('https://storage.googleapis.com/aulas-bc26-dados-gbs/cenipa/fator_contribuinte.csv', sep=';')
df2 = pd.read_csv('https://storage.googleapis.com/aulas-bc26-dados-gbs/cenipa/aeronave.csv', sep=';')
df3 = pd.read_csv('https://storage.googleapis.com/aulas-bc26-dados-gbs/cenipa/ocorrencia_tipo.csv', sep=';')
df4 = pd.read_csv('https://storage.googleapis.com/aulas-bc26-dados-gbs/cenipa/recomendacao.csv', sep=';')

In [ ]:
# backup dos dados
dfback = df.copy()
dfback1 = df1.copy()
dfback2 = df2.copy()
dfback3 = df3.copy()
dfback4 = df4.copy()

## Tratamento df (ocorrencia.csv)

In [ ]:
# drop colunas repetidas da df
df.drop(['codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4'], axis=1, inplace=True)
# drop colunas que não serão utilizadas da df
df.drop(['ocorrencia_latitude','ocorrencia_longitude','investigacao_aeronave_liberada','investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado','divulgacao_dia_publicacao'], axis=1, inplace=True)

In [ ]:
# trasformando type para datetime
# pd.to_datetime(df['ocorrencia_dia'], dayfirst=True)

# drop de linha
#df.drop([6768], axis=0, inplace=True)

In [ ]:
df.dtypes

codigo_ocorrencia                      int64
ocorrencia_classificacao              object
ocorrencia_cidade                     object
ocorrencia_uf                         object
ocorrencia_pais                       object
ocorrencia_aerodromo                  object
ocorrencia_dia                datetime64[ns]
ocorrencia_hora                       object
total_recomendacoes                    int64
total_aeronaves_envolvidas             int64
ocorrencia_saida_pista                object
dtype: object

In [ ]:
# renomear as colunas da df
df.rename(columns={
    'codigo_ocorrencia':'ocorrencia',
    'ocorrencia_classificacao':'classificacao',
    'ocorrencia_cidade':'cidade',
    'ocorrencia_uf':'uf',
    'ocorrencia_pais':'pais',
    'ocorrencia_aerodromo':'aerodromo',
    'ocorrencia_dia':'dia',
    'ocorrencia_hora':'horario',
    'total_recomendacoes':'recomendacoes',
    'total_aeronaves_envolvidas':'n_aeronaves',
    'ocorrencia_saida_pista':'saida_pista'
    }, inplace=True)

In [ ]:
df.dtypes

ocorrencia                int64
classificacao            object
cidade                   object
uf                       object
pais                     object
aerodromo                object
dia              datetime64[ns]
horario                  object
recomendacoes             int64
n_aeronaves               int64
saida_pista              object
dtype: object

In [ ]:
df.head(5)

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM
1,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO
2,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM
3,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,SBBH,2022-10-06,00:44:00,0,1,NÃO
4,81025,INCIDENTE,MANAUS,AM,BRASIL,SBEG,2022-10-05,20:56:00,0,1,NÃO


In [ ]:
# Dados nulos antes do tratamento
# Retorna: Null NaN NaT <NaN>...
df.isnull().sum()

ocorrencia       0
classificacao    0
cidade           0
uf               0
pais             0
aerodromo        0
dia              0
horario          2
recomendacoes    0
n_aeronaves      0
saida_pista      0
dtype: int64

In [ ]:
# .unique()
sorted(df['aerodromo'].unique())
# inconcistências encontradas: ***, **NI

['***',
 '**NI',
 '5JTI',
 '9PAA',
 '9PBS',
 '9PFC',
 '9PFX',
 '9PHD',
 '9PSD',
 'APUA',
 'SBAE',
 'SBAN',
 'SBAQ',
 'SBAR',
 'SBAT',
 'SBAU',
 'SBAX',
 'SBBE',
 'SBBH',
 'SBBI',
 'SBBP',
 'SBBQ',
 'SBBR',
 'SBBU',
 'SBBV',
 'SBBW',
 'SBBZ',
 'SBCA',
 'SBCB',
 'SBCF',
 'SBCG',
 'SBCH',
 'SBCJ',
 'SBCP',
 'SBCR',
 'SBCT',
 'SBCX',
 'SBCY',
 'SBCZ',
 'SBDN',
 'SBEG',
 'SBEK',
 'SBFE',
 'SBFI',
 'SBFL',
 'SBFN',
 'SBFS',
 'SBFZ',
 'SBGL',
 'SBGO',
 'SBGP',
 'SBGR',
 'SBGV',
 'SBGW',
 'SBHT',
 'SBIH',
 'SBIL',
 'SBIP',
 'SBIT',
 'SBIZ',
 'SBJA',
 'SBJC',
 'SBJD',
 'SBJF',
 'SBJH',
 'SBJI',
 'SBJP',
 'SBJR',
 'SBJU',
 'SBJV',
 'SBKG',
 'SBKP',
 'SBLE',
 'SBLJ',
 'SBLO',
 'SBMA',
 'SBMD',
 'SBME',
 'SBMG',
 'SBMK',
 'SBML',
 'SBMO',
 'SBMQ',
 'SBMS',
 'SBMT',
 'SBMY',
 'SBNF',
 'SBNM',
 'SBNT',
 'SBNV',
 'SBPA',
 'SBPB',
 'SBPC',
 'SBPF',
 'SBPG',
 'SBPJ',
 'SBPK',
 'SBPL',
 'SBPN',
 'SBPO',
 'SBPP',
 'SBPR',
 'SBPS',
 'SBPV',
 'SBQV',
 'SBRB',
 'SBRD',
 'SBRF',
 'SBRJ',
 'SBRP',
 'SBSC',
 '

In [ ]:
# Tratamento de inconsistências
df.replace(['***','**NI'],pd.NA, inplace = True)
#dataFrame.replace('valor_para_alterar','Valor_de_alteração', inplace=True(salvar))

In [ ]:
# após o tratamento
df.isnull().sum()

ocorrencia          0
classificacao       0
cidade              0
uf                  4
pais                0
aerodromo        2618
dia                 0
horario             2
recomendacoes       0
n_aeronaves         0
saida_pista         0
dtype: int64

### Validação df

In [ ]:
# criação do schema de validação dos dados

schema = pa.DataFrameSchema(
      columns = {
          'classificacao':pa.Column(pa.String),
          'ocorrencia':pa.Column(pa.Int),
          'cidade':pa.Column(pa.String),
          'uf':pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
          'pais':pa.Column(pa.String),
          'aerodromo':pa.Column(pa.String,pa.Check.str_length(4,4),nullable=True),
          'dia':pa.Column(pa.DateTime),
          'horario':pa.Column(pa.String,pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'),nullable=True),
          'recomendacoes':pa.Column(pa.Int),
          'n_aeronaves':pa.Column(pa.Int),
          'saida_pista':pa.Column(pa.String,pa.Check.str_length(3,3)),
      }
  )

In [ ]:
#validador do DF
schema.validate(df)

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM
1,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO
2,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM
3,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,SBBH,2022-10-06,00:44:00,0,1,NÃO
4,81025,INCIDENTE,MANAUS,AM,BRASIL,SBEG,2022-10-05,20:56:00,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...
6764,40324,INCIDENTE,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,0,1,NÃO
6765,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,BRASIL,<NA>,2010-01-04,17:30:00,0,1,NÃO
6766,40211,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,<NA>,2010-01-03,12:00:00,0,1,NÃO
6767,40349,INCIDENTE,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,0,1,NÃO


## Tratamento df1 (fator_contribuinte.csv)

In [ ]:
df1.dtypes

codigo_ocorrencia3      int64
fator_nome             object
fator_aspecto          object
fator_condicionante    object
fator_area             object
dtype: object

In [ ]:
# renomear as colunas da df1
df1.rename(columns={'codigo_ocorrencia3':'ocorrencia'}, inplace = True)

In [ ]:
df1.dtypes

ocorrencia              int64
fator_nome             object
fator_aspecto          object
fator_condicionante    object
fator_area             object
dtype: object

In [ ]:
df1.head(5)

,ocorrencia,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,80383,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
1,80383,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
2,80383,PLANEJAMENTO DE VOO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
3,80383,SUPERVISÃO GERENCIAL,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
4,80365,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO


In [ ]:
df1.isnull().sum()

ocorrencia             0
fator_nome             0
fator_aspecto          0
fator_condicionante    0
fator_area             0
dtype: int64

In [ ]:
df1.groupby(['fator_condicionante']).size()
# inconsistências encontradas: '***'

fator_condicionante
***                                        489
INDIVIDUAL                                 973
MANUTENÇÃO DA AERONAVE                     274
OPERAÇÃO DA AERONAVE                      2454
ORGANIZACIONAL                             410
PRESTAÇÃO DE SERVIÇOS DE TRÁFEGO AÉREO      29
PSICOSSOCIAL                               147
dtype: int64

In [ ]:
# tratando inconsistencia em fator_condicionante
df1.fator_condicionante.replace('***',pd.NA, inplace = True)

In [ ]:
df1

,ocorrencia,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,80383,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
1,80383,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
2,80383,PLANEJAMENTO DE VOO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
3,80383,SUPERVISÃO GERENCIAL,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
4,80365,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
...,...,...,...,...,...
4771,43657,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
4772,43628,FADIGA,ASPECTO MÉDICO,<NA>,FATOR HUMANO
4773,43628,MEMÓRIA,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
4774,43628,ORGANIZAÇÃO DO TRABALHO,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO


## Tratamento df2 (aeronave.csv)

In [ ]:
# renomear as colunas da df2
df2.rename(columns={'codigo_ocorrencia2':'ocorrencia'}, inplace=True)

In [ ]:
df2.dtypes

ocorrencia                       int64
aeronave_matricula              object
aeronave_operador_categoria     object
aeronave_tipo_veiculo           object
aeronave_fabricante             object
aeronave_modelo                 object
aeronave_tipo_icao              object
aeronave_motor_tipo             object
aeronave_motor_quantidade       object
aeronave_pmd                   float64
aeronave_pmd_categoria         float64
aeronave_assentos              float64
aeronave_ano_fabricacao        float64
aeronave_pais_fabricante        object
aeronave_pais_registro          object
aeronave_registro_categoria     object
aeronave_registro_segmento      object
aeronave_voo_origem             object
aeronave_voo_destino            object
aeronave_fase_operacao          object
aeronave_tipo_operacao          object
aeronave_nivel_dano             object
aeronave_fatalidades_total     float64
dtype: object

In [ ]:
df2.head(5)

,ocorrencia,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,43628,PTEHG,***,AVIÃO,EMBRAER,EMB-820C NAVAJO,PA31,PISTÃO,BIMOTOR,3175.0,3175.0,10.0,1976.0,BRASIL,BRASIL,AVIÃO,TÁXI AÉREO,DEPUTADO LUÍS EDUARDO MAGALHÃES,FAZENDA PONTAL,POUSO,TÁXI AÉREO,SUBSTANCIAL,0.0
1,43629,PTHVW,***,HELICÓPTERO,ROBINSON HELICOPTER,R22 BETA,R22,PISTÃO,MONOMOTOR,621.0,621.0,2.0,1991.0,BRASIL,BRASIL,HELICÓPTERO,INSTRUÇÃO,FORA DE AERODROMO,FORA DE AERODROMO,MANOBRA,INSTRUÇÃO,SUBSTANCIAL,0.0
2,43630,PTXRK,ESPECIALIZADA,AVIÃO,AIR TRACTOR,AT-401B,AT3P,PISTÃO,MONOMOTOR,2722.0,2722.0,1.0,1994.0,BRASIL,BRASIL,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,ESPECIALIZADA,ESPECIALIZADA,LEVE,0.0
3,43631,PRGGM,REGULAR,AVIÃO,BOEING COMPANY,737-8EH,B738,JATO,BIMOTOR,70533.0,70533.0,197.0,2009.0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DESCIDA,REGULAR,NENHUM,0.0
4,43633,PRPSK,***,AVIÃO,EMBRAER,EMB-145LR,E145,JATO,BIMOTOR,21990.0,21990.0,54.0,2001.0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,REGULAR,NENHUM,0.0


In [ ]:
# verificar: aeronave_pais_fabricante	aeronave_pais_registro != BRASIL
# as duas tem valores diferentes
df2.aeronave_pais_fabricante.unique()
# inconsistencia encontrada: 'nan'

array(['BRASIL', 'ESTADOS UNIDOS', nan, 'PORTUGAL', 'ESPANHA',
       'ÁFRICA DO SUL', 'PARAGUAI', 'BOLÍVIA', 'CHILE', 'SUIÇA',
       'ALEMANHA', 'COLÔMBIA', 'URUGUAI', 'CINGAPURA', 'FRANÇA', 'RÚSSIA',
       'POLÔNIA', 'EMIRADOS ÁRABES', 'PANAMÁ', 'ITÁLIA', 'VENEZUELA',
       'ARGENTINA', 'NÃO IDENTIFICADO', 'HOLANDA', 'UCRÂNIA'],
      dtype=object)

In [ ]:
df2.aeronave_pais_fabricante.replace('nan',pd.NA, inplace = True)

In [ ]:
# verificando se as colunas aeronave_pmd e aeronave_pmd_categoria são iguais 
df2['aeronave_pmd'].equals(df2['aeronave_pmd_categoria'])

True

In [ ]:
# verificando se as colunas aeronave_pais_fabricante e aeronave_pais_registro são iguais
df2['aeronave_pais_fabricante'].equals(df2['aeronave_pais_registro'])

True

In [ ]:
# drop em coluna com dados repetidos
df2.drop(columns=['aeronave_pais_registro','aeronave_pmd_categoria'], axis = 1, inplace = True)

In [ ]:
df2.head(5)

,ocorrencia,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,43628,PTEHG,***,AVIÃO,EMBRAER,EMB-820C NAVAJO,PA31,PISTÃO,BIMOTOR,3175.0,10.0,1976.0,BRASIL,AVIÃO,TÁXI AÉREO,DEPUTADO LUÍS EDUARDO MAGALHÃES,FAZENDA PONTAL,POUSO,TÁXI AÉREO,SUBSTANCIAL,0.0
1,43629,PTHVW,***,HELICÓPTERO,ROBINSON HELICOPTER,R22 BETA,R22,PISTÃO,MONOMOTOR,621.0,2.0,1991.0,BRASIL,HELICÓPTERO,INSTRUÇÃO,FORA DE AERODROMO,FORA DE AERODROMO,MANOBRA,INSTRUÇÃO,SUBSTANCIAL,0.0
2,43630,PTXRK,ESPECIALIZADA,AVIÃO,AIR TRACTOR,AT-401B,AT3P,PISTÃO,MONOMOTOR,2722.0,1.0,1994.0,BRASIL,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,ESPECIALIZADA,ESPECIALIZADA,LEVE,0.0
3,43631,PRGGM,REGULAR,AVIÃO,BOEING COMPANY,737-8EH,B738,JATO,BIMOTOR,70533.0,197.0,2009.0,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DESCIDA,REGULAR,NENHUM,0.0
4,43633,PRPSK,***,AVIÃO,EMBRAER,EMB-145LR,E145,JATO,BIMOTOR,21990.0,54.0,2001.0,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,REGULAR,NENHUM,0.0


In [ ]:
#alterando o restante dos nomes das colunas
df2.rename(columns={'aeronave_matricula':'aero_matricula',
                    'aeronave_operador_categoria':'categoria_op',
                    'aeronave_tipo_veiculo':'tipo_aeronave',
                    'aeronave_fabricante':'fabricante',
                    'aeronave_modelo':'modelo',
                    'aeronave_tipo_icao':'aero_tipo_icao',
                    'aeronave_motor_tipo':'tipo_motor',	
                    'aeronave_motor_quantidade':'qnt_motor',
                    'aeronave_pmd':'aero_pmd',
                    'aeronave_assentos':'n_assentos',
                    'aeronave_ano_fabricacao':'ano_fab',
                    'aeronave_pais_fabricante':'pais_fab',
                    'aeronave_registro_categoria':'cat_reg',
                    'aeronave_registro_segmento':'reg_seg',
                    'aeronave_voo_origem':'voo_origem',
                    'aeronave_voo_destino':'voo_destino',
                    'aeronave_fase_operacao':'fase_op',
                    'aeronave_tipo_operacao':'tipo_op',
                    'aeronave_nivel_dano':'nivel_dano',
                    'aeronave_fatalidades_total':'total_mortes'  
}, inplace = True)

In [ ]:
df2.dtypes

ocorrencia          int64
aero_matricula     object
categoria_op       object
tipo_aeronave      object
fabricante         object
modelo             object
aero_tipo_icao     object
tipo_motor         object
qnt_motor          object
aero_pmd          float64
n_assentos        float64
ano_fab           float64
pais_fab           object
cat_reg            object
reg_seg            object
voo_origem         object
voo_destino        object
fase_op            object
tipo_op            object
nivel_dano         object
total_mortes      float64
dtype: object

In [ ]:
# verificando inconsistencias
df2.isnull().sum()

ocorrencia          0
aero_matricula     33
categoria_op       33
tipo_aeronave      33
fabricante         33
modelo             33
aero_tipo_icao     33
tipo_motor         50
qnt_motor          33
aero_pmd           33
n_assentos        275
ano_fab           261
pais_fab           33
cat_reg            33
reg_seg            33
voo_origem         34
voo_destino        34
fase_op            33
tipo_op            33
nivel_dano         33
total_mortes       33
dtype: int64

In [ ]:
# .unique()
df2['aero_tipo_icao'].unique()
# inconcistências encontradas: ***, **NI, *

array(['PA31', 'R22', 'AT3P', 'B738', 'E145', 'AT43', 'DC10', 'B350',
       'AB11', 'A122', 'PA34', 'A320', 'P28A', 'S76', 'A139', 'H269',
       'C188', 'PA25', nan, 'C208', 'IPAN', 'BE20', 'P32R', 'AT72',
       'BE58', 'C182', 'PRM1', 'AS50', 'BE36', 'AT45', 'C150', 'A310',
       'P28R', 'SR22', 'B722', 'C206', 'C170', 'E190', 'BE9L', 'LJ31',
       'ULAC', 'PAUL', 'F100', 'A321', 'PAY1', 'PA32', 'R44', 'C152',
       'B733', 'BE55', 'B737', 'A109', '***', 'C650', 'E170', 'LJ35',
       'AC90', 'B407', 'BE33', 'P28T', 'A319', 'C210', 'C550', 'B06',
       'AS55', 'A332', 'PAT4', 'SR20', 'AT5T', 'MU2', 'LJ45', 'C25A',
       'PA18', 'C172', 'AT8T', 'AS65', 'GLID', 'S92', 'EC20', 'PA46',
       'BN2P', 'A119', 'E110', 'P28B', 'EC25', 'S61', 'L410', 'BE17',
       'C72R', 'PA36', 'E50P', 'PC12', 'E121', 'ZZZZ', 'B763', 'PAY2',
       'EC55', 'PA30', 'C77R', 'AB18', 'BE40', 'B212', 'C310', 'BE35',
       'LA25', 'TBM7', 'E55P', 'B105', 'A340', 'B77W', 'P210', 'S108',
       'BE9T', 'C

In [ ]:
# tratamento de inconsistências
df2.replace(['***','**NI', '*'],pd.NA,inplace=True)

In [ ]:
# verificando inconsistencias após alteração
df2.isnull().sum()

ocorrencia           0
aero_matricula      33
categoria_op      3725
tipo_aeronave      201
fabricante         433
modelo             218
aero_tipo_icao     329
tipo_motor         322
qnt_motor          139
aero_pmd            33
n_assentos         275
ano_fab            261
pais_fab            33
cat_reg            201
reg_seg            115
voo_origem          34
voo_destino         34
fase_op             61
tipo_op            160
nivel_dano          89
total_mortes        33
dtype: int64

In [ ]:
df2

,ocorrencia,aero_matricula,categoria_op,tipo_aeronave,fabricante,modelo,aero_tipo_icao,tipo_motor,qnt_motor,aero_pmd,n_assentos,ano_fab,pais_fab,cat_reg,reg_seg,voo_origem,voo_destino,fase_op,tipo_op,nivel_dano,total_mortes
0,43628,PTEHG,<NA>,AVIÃO,EMBRAER,EMB-820C NAVAJO,PA31,PISTÃO,BIMOTOR,3175.0,10.0,1976.0,BRASIL,AVIÃO,TÁXI AÉREO,DEPUTADO LUÍS EDUARDO MAGALHÃES,FAZENDA PONTAL,POUSO,TÁXI AÉREO,SUBSTANCIAL,0.0
1,43629,PTHVW,<NA>,HELICÓPTERO,ROBINSON HELICOPTER,R22 BETA,R22,PISTÃO,MONOMOTOR,621.0,2.0,1991.0,BRASIL,HELICÓPTERO,INSTRUÇÃO,FORA DE AERODROMO,FORA DE AERODROMO,MANOBRA,INSTRUÇÃO,SUBSTANCIAL,0.0
2,43630,PTXRK,ESPECIALIZADA,AVIÃO,AIR TRACTOR,AT-401B,AT3P,PISTÃO,MONOMOTOR,2722.0,1.0,1994.0,BRASIL,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,ESPECIALIZADA,ESPECIALIZADA,LEVE,0.0
3,43631,PRGGM,REGULAR,AVIÃO,BOEING COMPANY,737-8EH,B738,JATO,BIMOTOR,70533.0,197.0,2009.0,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DESCIDA,REGULAR,NENHUM,0.0
4,43633,PRPSK,<NA>,AVIÃO,EMBRAER,EMB-145LR,E145,JATO,BIMOTOR,21990.0,54.0,2001.0,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,REGULAR,NENHUM,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6334,81025,PTWRU,<NA>,ANFÍBIO,CESSNA AIRCRAFT,208 ANFIBIO,C208,TURBOÉLICE,MONOMOTOR,3629.0,10.0,1998.0,BRASIL,ANFÍBIO,TÁXI AÉREO,EDUARDO GOMES,EDUARDO GOMES,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0
6335,81026,PR-YRR,<NA>,AVIÃO,AIRBUS,A320-253N,<NA>,JATO,SEM TRAÇÃO,77000.0,0.0,0.0,BRASIL,AVIÃO,REGULAR,SANTOS DUMONT,DEPUTADO LUÍS EDUARDO MAGALHÃES,DECOLAGEM,REGULAR,NENHUM,0.0
6336,81027,PPMIX,<NA>,AVIÃO,BOMBARDIER,LEARJET 45(75),<NA>,JATO,SEM TRAÇÃO,9752.0,0.0,0.0,BRASIL,AVIÃO,PARTICULAR,CATARATAS,CONGONHAS,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0
6337,81029,PTVMB,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-810D,PA34,PISTÃO,BIMOTOR,2155.0,4.0,1990.0,BRASIL,AVIÃO,PARTICULAR,JACAREPAGUÁ,CARLOS DRUMMOND DE ANDRADE / PAMPULHA,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0


## Tratamento df3 (ocorrencia_tipo.csv)

In [ ]:
df3.dtypes

codigo_ocorrencia1            int64
ocorrencia_tipo              object
ocorrencia_tipo_categoria    object
taxonomia_tipo_icao          object
dtype: object

In [ ]:
# renomeando colunas
df3.rename(columns={'codigo_ocorrencia1':'ocorrencia',
                    'ocorrencia_tipo':'tipo_oc',
                    'ocorrencia_tipo_categoria':'tipo_cat_oc',
                    'taxonomia_tipo_icao':'taxonomia_tipo_icao'}, inplace = True)

In [ ]:
df3.dtypes

ocorrencia              int64
tipo_oc                object
tipo_cat_oc            object
taxonomia_tipo_icao    object
dtype: object

In [ ]:
df3.head(5)

,ocorrencia,tipo_oc,tipo_cat_oc,taxonomia_tipo_icao
0,81030,DESCOMPRESSÃO NÃO INTENCIONAL / EXPLOSIVA,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
1,81029,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,81027,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,81027,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
4,81026,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP


In [ ]:
df3.isnull().sum()

ocorrencia             0
tipo_oc                1
tipo_cat_oc            1
taxonomia_tipo_icao    1
dtype: int64

In [ ]:
# verificando inconsistências
df3.groupby(['tipo_oc']).size()

tipo_oc
***                                             11
AERONAVE ATINGIDA POR OBJETO                    16
AERÓDROMO                                       12
ALARME FALSO DE FOGO OU DE SUPERAQUECIMENTO     19
CAUSADO POR FENÔMENO METEOROLÓGICO EM VOO      107
                                              ... 
TRÁFEGO AÉREO                                  127
TURBULÊNCIA                                      8
VAZAMENTO DE COMBUSTÍVEL                        20
VAZAMENTO DE OUTROS FLUIDOS                     59
VOO CONTROLADO CONTRA O TERRENO                 44
Length: 83, dtype: int64

In [ ]:
df3.groupby(['tipo_cat_oc']).size()

tipo_cat_oc
***                                                  11
*** COLISÃO EM VOO COM OBSTÁCULO (excluir)            2
*** SAÍDA DE PISTA - (excluir)                        1
AERÓDROMO                                            12
COLISÃO COM AVE                                     340
                                                   ... 
POUSO AQUÉM/ALÉM DA PISTA | POUSO ANTES DA PISTA     18
REBOQUE DE PLANADOR                                   2
RELACIONADO COM SECURITY                              1
TURBULÊNCIA                                           8
VOO CONTROLADO CONTRA O TERRENO                      44
Length: 83, dtype: int64

In [ ]:
df3.groupby(['taxonomia_tipo_icao']).size()

taxonomia_tipo_icao
***          11
ADRM         12
AMAN         16
ARC         242
ATM/CNS      15
BIRD        340
CFIT         44
CTOL        205
EVAC          1
EXCLUIR       3
EXTL          9
F-NI         70
FUEL        133
GCOL        142
GTOW          2
ICE           4
LALT        117
LOC-G       518
LOC-I       425
LOLI          2
MAC         140
OTHR        787
RE          368
RI           38
SCF-NP     2227
SCF-PP      987
SEC           1
TURB          8
UIMC          6
UNK         154
USOS         20
WILD         27
WSTRW        12
[RAMP]       13
dtype: int64

In [ ]:
df3.replace('***',pd.NA, inplace = True)

In [ ]:
df3.isnull().sum()

ocorrencia              0
tipo_oc                12
tipo_cat_oc            12
taxonomia_tipo_icao    12
dtype: int64

In [ ]:
df3

,ocorrencia,tipo_oc,tipo_cat_oc,taxonomia_tipo_icao
0,81030,DESCOMPRESSÃO NÃO INTENCIONAL / EXPLOSIVA,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
1,81029,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,81027,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,81027,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
4,81026,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
...,...,...,...,...
7095,39176,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
7096,39158,SUPERAQUECIMENTO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | SUPERAQU...,SCF-PP
7097,39156,FOGO EM VOO,FOGO/FUMAÇA (SEM IMPACTO) | FOGO EM VOO,F-NI
7098,39155,VAZAMENTO DE COMBUSTÍVEL,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP


## Tratamento df4 (recomendacao.csv)

In [ ]:
df4.dtypes

codigo_ocorrencia4                  int64
recomendacao_numero                object
recomendacao_dia_assinatura        object
recomendacao_dia_encaminhamento    object
recomendacao_dia_feedback          object
recomendacao_conteudo              object
recomendacao_status                object
recomendacao_destinatario_sigla    object
recomendacao_destinatario          object
dtype: object

In [ ]:
# renomear as colunas da df4
df4.rename(columns={'codigo_ocorrencia4':'ocorrencia',
                    'recomendacao_numero':'num_recomend',
                    'recomendacao_dia_assinatura':'dia_assinatura_recomend',
                    'recomendacao_dia_encaminhamento':'dia_encaminhamento_recomend',
                    'recomendacao_dia_feedback':'dia_feedback_recomend',
                    'recomendacao_conteudo':'recomendacao',
                    'recomendacao_status':'status_recomend',
                    'recomendacao_destinatario_sigla':'sigla_destinatario',
                    'recomendacao_destinatario':'destinatario'}, inplace = True)

In [ ]:
df4.dtypes

ocorrencia                      int64
num_recomend                   object
dia_assinatura_recomend        object
dia_encaminhamento_recomend    object
dia_feedback_recomend          object
recomendacao                   object
status_recomend                object
sigla_destinatario             object
destinatario                   object
dtype: object

In [ ]:
df4.head(5)

,ocorrencia,num_recomend,dia_assinatura_recomend,dia_encaminhamento_recomend,dia_feedback_recomend,recomendacao,status_recomend,sigla_destinatario,destinatario
0,80258,A-137/CENIPA/2019 - 01,2021-11-16,2021-12-23,2022-01-26,Atuar junto ao operador do Aeródromo de Tangar...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,80252,A-094/CENIPA/2021 - 01,2022-09-21,2022-09-27,NaN,Atuar junto ao proprietário da área não cadast...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,80073,A-063/CENIPA/2021 - 01,2021-11-12,2021-11-19,2021-12-02,Divulgar os ensinamentos colhidos nesta invest...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,80073,A-063/CENIPA/2021 - 02,2021-11-12,2021-11-19,2021-12-02,Divulgar os ensinamentos colhidos nesta invest...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,79997,A-050/CENIPA/2021 - 01,2022-08-05,2022-08-11,NaN,Reforçar a atuação junto à Pulveriza Aviação A...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


In [ ]:
df4.isnull().sum()

ocorrencia                       0
num_recomend                     0
dia_assinatura_recomend          0
dia_encaminhamento_recomend      0
dia_feedback_recomend          431
recomendacao                     4
status_recomend                  0
sigla_destinatario               0
destinatario                     0
dtype: int64

In [ ]:
# inconsistencia '***' localizada 
df4.groupby(['status_recomend']).size()

status_recomend
***                                   168
AGUARDANDO RESPOSTA                   113
IMPLEMENTADA                         1585
IMPLEMENTADA DE FORMA ALTERNATIVA     128
NÃO IMPLEMENTADA                      126
dtype: int64

In [ ]:
df4.groupby(['destinatario']).size()

destinatario
AERO AGRÍCOLA SANTOS DUMONT                                                        1
AEROCLUBE DE BRAGANÇA PAULISTA                                                     1
AEROCLUBE DE GOIÁS                                                                 1
AEROCLUBE DE PONTA GROSSA                                                          1
AEROCLUBE DE PORTO NACIONAL                                                        2
                                                                                  ..
TEMA TECNOLOGIA MANUTENÇÃO DE AERONAVES                                            4
TERCEIRO SERVIÇO REGIONAL DE INVESTIGAÇÃO E PREVENÇÃO DE ACIDENTES AERONÁUTICOS    4
TSB                                                                                1
TÁXI AÉREO SUL DO PARÁ                                                             1
VIAÇÃO AÉREA RIO GRANDENSE                                                         1
Length: 88, dtype: int64

In [ ]:
df4.replace('***',pd.NA, inplace = True)

In [ ]:
df4

,ocorrencia,num_recomend,dia_assinatura_recomend,dia_encaminhamento_recomend,dia_feedback_recomend,recomendacao,status_recomend,sigla_destinatario,destinatario
0,80258,A-137/CENIPA/2019 - 01,2021-11-16,2021-12-23,2022-01-26,Atuar junto ao operador do Aeródromo de Tangar...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,80252,A-094/CENIPA/2021 - 01,2022-09-21,2022-09-27,NaN,Atuar junto ao proprietário da área não cadast...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,80073,A-063/CENIPA/2021 - 01,2021-11-12,2021-11-19,2021-12-02,Divulgar os ensinamentos colhidos nesta invest...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,80073,A-063/CENIPA/2021 - 02,2021-11-12,2021-11-19,2021-12-02,Divulgar os ensinamentos colhidos nesta invest...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,79997,A-050/CENIPA/2021 - 01,2022-08-05,2022-08-11,NaN,Reforçar a atuação junto à Pulveriza Aviação A...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
...,...,...,...,...,...,...,...,...,...
2115,39235,207/2011,2011-08-29,2011-08-29,NaN,Reforçar junto aos seus mecânicos a necessidad...,IMPLEMENTADA,SERIPA-6,SEXTO SERVIÇO REGIONAL DE INVESTIGAÇÃO E PREVE...
2116,39156,063/2011,2011-06-30,2011-08-01,2011-09-12,Divulgar este Relatório aos operadores de aero...,IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2117,39156,064/2011,2011-06-30,2011-08-01,2014-04-16,Acompanhar junto à autoridade primária de cert...,IMPLEMENTADA DE FORMA ALTERNATIVA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2118,39115,095/2011,2011-07-21,2011-08-01,2015-12-21,Revisar os requisitos de experiência recente p...,NÃO IMPLEMENTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


In [ ]:
df4.dtypes

ocorrencia                      int64
num_recomend                   object
dia_assinatura_recomend        object
dia_encaminhamento_recomend    object
dia_feedback_recomend          object
recomendacao                   object
status_recomend                object
sigla_destinatario             object
destinatario                   object
dtype: object

In [ ]:
#drop colunas que não serão utilizadas da df4
df4.drop(['dia_assinatura_recomend', 'dia_encaminhamento_recomend', 'dia_feedback_recomend'], axis=1, inplace=True)
#drop colunas que repetidas da df4
df4.drop(['sigla_destinatario'], axis=1, inplace=True)

In [ ]:
df4.head(5)

,ocorrencia,num_recomend,recomendacao,status_recomend,destinatario
0,80258,A-137/CENIPA/2019 - 01,Atuar junto ao operador do Aeródromo de Tangar...,IMPLEMENTADA,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,80252,A-094/CENIPA/2021 - 01,Atuar junto ao proprietário da área não cadast...,AGUARDANDO RESPOSTA,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,80073,A-063/CENIPA/2021 - 01,Divulgar os ensinamentos colhidos nesta invest...,IMPLEMENTADA,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,80073,A-063/CENIPA/2021 - 02,Divulgar os ensinamentos colhidos nesta invest...,IMPLEMENTADA,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,79997,A-050/CENIPA/2021 - 01,Reforçar a atuação junto à Pulveriza Aviação A...,AGUARDANDO RESPOSTA,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


## Merge dff1

In [ ]:
#criando data frame agregado
#outer = o que encontrar de relação ele adiciona, e o que não encontrar ele repete 
#inner = usar quando você quer todas as colunas preenchidas
dff1 = df.merge(df1, how='outer')
dff1.head(5)

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN
1,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO,NaN,NaN,NaN,NaN
2,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM,NaN,NaN,NaN,NaN
3,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,SBBH,2022-10-06,00:44:00,0,1,NÃO,NaN,NaN,NaN,NaN
4,81025,INCIDENTE,MANAUS,AM,BRASIL,SBEG,2022-10-05,20:56:00,0,1,NÃO,NaN,NaN,NaN,NaN


## Merge ddf2

In [ ]:
#criando data frame agregado
#outer = o que encontrar de relação ele adiciona, e o que não encontrar ele repete 
#inner = usar quando você quer todas as colunas preenchidas
dff2 = dff1.merge(df2, how='outer')
dff2.head(5)

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista,fator_nome,fator_aspecto,fator_condicionante,fator_area,aero_matricula,categoria_op,tipo_aeronave,fabricante,modelo,aero_tipo_icao,tipo_motor,qnt_motor,aero_pmd,n_assentos,ano_fab,pais_fab,cat_reg,reg_seg,voo_origem,voo_destino,fase_op,tipo_op,nivel_dano,total_mortes
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPMIX,<NA>,AVIÃO,BOMBARDIER,LEARJET 45(75),<NA>,JATO,SEM TRAÇÃO,9752.0,0.0,0.0,BRASIL,AVIÃO,PARTICULAR,CATARATAS,CONGONHAS,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0
1,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRAUQ,<NA>,AVIÃO,EMBRAER,ERJ 190-200 IGW,E190,JATO,BIMOTOR,52290.0,125.0,2015.0,BRASIL,AVIÃO,REGULAR,TANCREDO NEVES,EURICO DE AGUIAR SALLES,CRUZEIRO,REGULAR,NENHUM,0.0
2,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPGSV,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660.0,2.0,1960.0,BRASIL,AVIÃO,INSTRUÇÃO,AMERICANA,AERODRÓMO ESTADUAL DE CAMPINAS / AMARAIS,DECOLAGEM,INSTRUÇÃO,LEVE,0.0
3,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,SBBH,2022-10-06,00:44:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTVMB,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-810D,PA34,PISTÃO,BIMOTOR,2155.0,4.0,1990.0,BRASIL,AVIÃO,PARTICULAR,JACAREPAGUÁ,CARLOS DRUMMOND DE ANDRADE / PAMPULHA,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0
4,81025,INCIDENTE,MANAUS,AM,BRASIL,SBEG,2022-10-05,20:56:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTWRU,<NA>,ANFÍBIO,CESSNA AIRCRAFT,208 ANFIBIO,C208,TURBOÉLICE,MONOMOTOR,3629.0,10.0,1998.0,BRASIL,ANFÍBIO,TÁXI AÉREO,EDUARDO GOMES,EDUARDO GOMES,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0


## Merge dff3

In [ ]:
dff3 = dff2.merge(df3, how='outer')
dff3.head(5)

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista,fator_nome,fator_aspecto,fator_condicionante,fator_area,aero_matricula,categoria_op,tipo_aeronave,fabricante,modelo,aero_tipo_icao,tipo_motor,qnt_motor,aero_pmd,n_assentos,ano_fab,pais_fab,cat_reg,reg_seg,voo_origem,voo_destino,fase_op,tipo_op,nivel_dano,total_mortes,tipo_oc,tipo_cat_oc,taxonomia_tipo_icao
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPMIX,<NA>,AVIÃO,BOMBARDIER,LEARJET 45(75),<NA>,JATO,SEM TRAÇÃO,9752.0,0.0,0.0,BRASIL,AVIÃO,PARTICULAR,CATARATAS,CONGONHAS,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
1,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPMIX,<NA>,AVIÃO,BOMBARDIER,LEARJET 45(75),<NA>,JATO,SEM TRAÇÃO,9752.0,0.0,0.0,BRASIL,AVIÃO,PARTICULAR,CATARATAS,CONGONHAS,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
2,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRAUQ,<NA>,AVIÃO,EMBRAER,ERJ 190-200 IGW,E190,JATO,BIMOTOR,52290.0,125.0,2015.0,BRASIL,AVIÃO,REGULAR,TANCREDO NEVES,EURICO DE AGUIAR SALLES,CRUZEIRO,REGULAR,NENHUM,0.0,DESCOMPRESSÃO NÃO INTENCIONAL / EXPLOSIVA,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPGSV,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660.0,2.0,1960.0,BRASIL,AVIÃO,INSTRUÇÃO,AMERICANA,AERODRÓMO ESTADUAL DE CAMPINAS / AMARAIS,DECOLAGEM,INSTRUÇÃO,LEVE,0.0,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
4,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPGSV,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660.0,2.0,1960.0,BRASIL,AVIÃO,INSTRUÇÃO,AMERICANA,AERODRÓMO ESTADUAL DE CAMPINAS / AMARAIS,DECOLAGEM,INSTRUÇÃO,LEVE,0.0,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G


## Merge dff4

In [ ]:
dff4 = dff3.merge(df4, how='outer')
dff4.head(5)

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista,fator_nome,fator_aspecto,fator_condicionante,fator_area,aero_matricula,categoria_op,tipo_aeronave,fabricante,modelo,aero_tipo_icao,tipo_motor,qnt_motor,aero_pmd,n_assentos,ano_fab,pais_fab,cat_reg,reg_seg,voo_origem,voo_destino,fase_op,tipo_op,nivel_dano,total_mortes,tipo_oc,tipo_cat_oc,taxonomia_tipo_icao,num_recomend,recomendacao,status_recomend,destinatario
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPMIX,<NA>,AVIÃO,BOMBARDIER,LEARJET 45(75),<NA>,JATO,SEM TRAÇÃO,9752.0,0.0,0.0,BRASIL,AVIÃO,PARTICULAR,CATARATAS,CONGONHAS,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
1,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPMIX,<NA>,AVIÃO,BOMBARDIER,LEARJET 45(75),<NA>,JATO,SEM TRAÇÃO,9752.0,0.0,0.0,BRASIL,AVIÃO,PARTICULAR,CATARATAS,CONGONHAS,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE,NaN,NaN,NaN,NaN
2,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRAUQ,<NA>,AVIÃO,EMBRAER,ERJ 190-200 IGW,E190,JATO,BIMOTOR,52290.0,125.0,2015.0,BRASIL,AVIÃO,REGULAR,TANCREDO NEVES,EURICO DE AGUIAR SALLES,CRUZEIRO,REGULAR,NENHUM,0.0,DESCOMPRESSÃO NÃO INTENCIONAL / EXPLOSIVA,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
3,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPGSV,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660.0,2.0,1960.0,BRASIL,AVIÃO,INSTRUÇÃO,AMERICANA,AERODRÓMO ESTADUAL DE CAMPINAS / AMARAIS,DECOLAGEM,INSTRUÇÃO,LEVE,0.0,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE,NaN,NaN,NaN,NaN
4,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PPGSV,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660.0,2.0,1960.0,BRASIL,AVIÃO,INSTRUÇÃO,AMERICANA,AERODRÓMO ESTADUAL DE CAMPINAS / AMARAIS,DECOLAGEM,INSTRUÇÃO,LEVE,0.0,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G,NaN,NaN,NaN,NaN


## Filtros

In [ ]:
ftclas = dff4.classificacao == 'INCIDENTE'
ftcentroeste = dff4.uf.isin(['MS', 'DF', 'MT', 'GO'])
ft2020 = dff4.dia.dt.year == 2020
dfac2020 = dff4.loc[ftclas & ft2020 & ftcentroeste]
dfac2020

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista,fator_nome,fator_aspecto,fator_condicionante,fator_area,aero_matricula,categoria_op,tipo_aeronave,fabricante,modelo,aero_tipo_icao,tipo_motor,qnt_motor,aero_pmd,n_assentos,ano_fab,pais_fab,cat_reg,reg_seg,voo_origem,voo_destino,fase_op,tipo_op,nivel_dano,total_mortes,tipo_oc,tipo_cat_oc,taxonomia_tipo_icao,num_recomend,recomendacao,status_recomend,destinatario
1248,79738,INCIDENTE,ITUMBIARA,GO,BRASIL,<NA>,2020-12-16,20:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTDZF,<NA>,AVIÃO,CESSNA AIRCRAFT,A188A,C188,PISTÃO,MONOMOTOR,1497.0,1.0,1971.0,BRASIL,AVIÃO,AGRÍCOLA,NÃO IDENTIFICADO,NÃO IDENTIFICADO,DECOLAGEM,AGRÍCOLA,LEVE,0.0,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,CTOL,NaN,NaN,NaN,NaN
1295,79750,INCIDENTE,GOIÂNIA,GO,BRASIL,SBNV,2020-12-01,11:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRBVT,<NA>,AVIÃO,PIPER AIRCRAFT,PA-31T1,PAY1,TURBOÉLICE,BIMOTOR,3946.0,7.0,1979.0,BRASIL,AVIÃO,TÁXI AÉREO,SANTA GENOVEVA/GOIÂNIA,AERÓDROMO NACIONAL DE AVIAÇÃO,CORRIDA APÓS POUSO,PRIVADA,LEVE,0.0,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
1306,79694,INCIDENTE,BRASÍLIA,DF,BRASIL,SBBR,2020-11-26,10:28:00,0,1,NÃO,NaN,NaN,NaN,NaN,PSAEC,<NA>,AVIÃO,EMBRAER,ERJ 190-400,<NA>,<NA>,SEM TRAÇÃO,0.0,0.0,0.0,BRASIL,AVIÃO,REGULAR,PRESIDENTE JUSCELINO KUBITSCHEK,VIRACOPOS,CRUZEIRO,REGULAR,NENHUM,0.0,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
1311,79689,INCIDENTE,QUERÊNCIA,MT,BRASIL,<NA>,2020-11-23,16:30:00,0,1,SIM,NaN,NaN,NaN,NaN,PRCMZ,<NA>,AVIÃO,THRUSH AIRCRAFT,S2R-H80,SS2T,TURBOÉLICE,MONOMOTOR,4763.0,NaN,NaN,BRASIL,AVIÃO,AGRÍCOLA,NÃO IDENTIFICADO,NÃO IDENTIFICADO,CORRIDA APÓS POUSO,AGRÍCOLA,LEVE,0.0,POUSO LONGO,EXCURSÃO DE PISTA | POUSO LONGO,RE,NaN,NaN,NaN,NaN
1314,79700,INCIDENTE,BRASÍLIA,DF,BRASIL,SBBR,2020-11-20,00:15:00,0,1,NÃO,NaN,NaN,NaN,NaN,PPPTP,<NA>,AVIÃO,AEROSPATIALE AND ALENIA,ATR-72-212A,AT72,TURBOÉLICE,BIMOTOR,22500.0,73.0,2009.0,BRASIL,AVIÃO,REGULAR,PRESIDENTE JUSCELINO KUBITSCHEK,PROFESSOR ERIBERTO MANOEL REINO,SUBIDA,REGULAR,NENHUM,0.0,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
1406,79571,INCIDENTE,BRASÍLIA,DF,BRASIL,SBBR,2020-09-27,22:39:00,0,1,NÃO,NaN,NaN,NaN,NaN,PPPTO,<NA>,AVIÃO,AEROSPATIALE AND ALENIA,ATR-72-212A,AT72,TURBOÉLICE,BIMOTOR,22500.0,73.0,2008.0,BRASIL,AVIÃO,REGULAR,TENENTE-CORONEL AVIADOR CÉSAR BOMBONATO,PRESIDENTE JUSCELINO KUBITSCHEK,POUSO,REGULAR,NENHUM,0.0,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
1432,79532,INCIDENTE,CUIABÁ,MT,BRASIL,SBCY,2020-09-10,12:53:00,0,1,SIM,NaN,NaN,NaN,NaN,PPOSP,<NA>,AVIÃO,CESSNA AIRCRAFT,208B,C208,TURBOÉLICE,MONOMOTOR,3969.0,11.0,2010.0,BRASIL,AVIÃO,REGULAR,FAZ. SANTA MARIA DO PORTO DE CAPIVARA,MARECHAL RONDON,CORRIDA APÓS POUSO,REGULAR,NENHUM,0.0,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,NaN,NaN,NaN,NaN
1433,79532,INCIDENTE,CUIABÁ,MT,BRASIL,SBCY,2020-09-10,12:53:00,0,1,SIM,NaN,NaN,NaN,NaN,PPOSP,<NA>,AVIÃO,CESSNA AIRCRAFT,208B,C208,TURBOÉLICE,MONOMOTOR,3969.0,11.0,2010.0,BRASIL,AVIÃO,REGULAR,FAZ. SANTA MARIA DO PORTO DE CAPIVARA,MARECHAL RONDON,CORRIDA APÓS POUSO,REGULAR,NENHUM,0.0,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE,NaN,NaN,NaN,NaN
1451,79525,INCIDENTE,VÁRZEA GRANDE,MT,BRASIL,SBCY,2020-09-04,23:10:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTIEC,<NA>,AVIÃO,TWIN COMMANDER,681,AC80,TURBOÉLICE,BIMOTOR,4264.0,7.0,1972.0,BRASIL,AVIÃO,TÁXI AÉREO,GOVERNADOR JORGE TEIXEIRA DE OLIVEIRA,MARECHAL RONDON,DESCIDA,TÁXI AÉREO,NENHUM,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
1458,79515,INCIDENTE,ARIPUANÃ,MT,BRASIL,<NA>,2020-08-31,15:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTMEJ,<NA>,AVIÃO,CESSNA AIRCRAFT,208B,C208,TURBOÉLICE,MONOMOTOR,3969

In [ ]:
dfac2020.groupby(['tipo_motor'], dropna=False).size().sort_values(ascending=False)

tipo_motor
TURBOÉLICE    16
PISTÃO         8
JATO           3
NaN            3
dtype: int64

In [ ]:
fttipoocorrencia = dff4.tipo_oc == 'FALHA DO MOTOR EM VOO'
ftcentroeste = dff4.uf.isin(['MS', 'DF', 'MT', 'GO'])
ft2021 = dff4.dia.dt.year == 2021
dfac2021 = dff4.loc[fttipoocorrencia & ft2021 & ftcentroeste]
dfac2021

,ocorrencia,classificacao,cidade,uf,pais,aerodromo,dia,horario,recomendacoes,n_aeronaves,saida_pista,fator_nome,fator_aspecto,fator_condicionante,fator_area,aero_matricula,categoria_op,tipo_aeronave,fabricante,modelo,aero_tipo_icao,tipo_motor,qnt_motor,aero_pmd,n_assentos,ano_fab,pais_fab,cat_reg,reg_seg,voo_origem,voo_destino,fase_op,tipo_op,nivel_dano,total_mortes,tipo_oc,tipo_cat_oc,taxonomia_tipo_icao,num_recomend,recomendacao,status_recomend,destinatario
598,80426,ACIDENTE,ANAURILÂNDIA,MS,BRASIL,<NA>,2021-12-11,20:00:00,0,1,NÃO,NaN,NaN,NaN,NaN,PUMDJ,<NA>,ULTRALEVE,<NA>,P2002 SIERRA DE LUXE,ZZZZ,PISTÃO,MONOMOTOR,600.0,NaN,NaN,BRASIL,ULTRALEVE,EXPERIMENTAL,NÃO IDENTIFICADO,NÃO IDENTIFICADO,APROXIMAÇÃO FINAL,PRIVADA,SUBSTANCIAL,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
628,80756,ACIDENTE,PARANATINGA,MT,BRASIL,<NA>,2021-11-28,20:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTUHE,<NA>,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-202,IPAN,PISTÃO,MONOMOTOR,1800.0,1.0,1992.0,BRASIL,AVIÃO,AGRÍCOLA,NÃO IDENTIFICADO,NÃO IDENTIFICADO,MANOBRA,ESPECIALIZADA,DESTRUÍDA,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
797,80237,ACIDENTE,ITAPORÃ,MS,BRASIL,SSPO,2021-08-10,21:00:00,0,1,NÃO,NaN,NaN,NaN,NaN,PPGAS,<NA>,AVIÃO,AERO BOERO,AB-115,AB11,PISTÃO,MONOMOTOR,770.0,2.0,1991.0,BRASIL,AVIÃO,INSTRUÇÃO,APLIC AVIAÇÃO AGRÍCOLA II,APLIC AVIAÇÃO AGRÍCOLA LTDA,DECOLAGEM,INSTRUÇÃO,SUBSTANCIAL,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
863,80181,INCIDENTE GRAVE,CASSILÂNDIA,MS,BRASIL,SSCL,2021-07-08,22:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PUPCA,<NA>,ULTRALEVE,<NA>,CONQUEST 210,ULAC,PISTÃO,MONOMOTOR,750.0,2.0,2007.0,BRASIL,ULTRALEVE,EXPERIMENTAL,CASSILÂNDIA,CASSILÂNDIA,APROXIMAÇÃO FINAL,EXPERIMENTAL,LEVE,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
898,80131,INCIDENTE GRAVE,BATAYPORÃ,MS,BRASIL,<NA>,2021-06-18,13:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRACK,<NA>,AVIÃO,CESSNA AIRCRAFT,A188B,C188,PISTÃO,MONOMOTOR,1905.0,1.0,1975.0,BRASIL,AVIÃO,AGRÍCOLA,FORA DE AERODROMO,FORA DE AERODROMO,MANOBRA,AGRÍCOLA,LEVE,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
905,80130,INCIDENTE GRAVE,GOIÂNIA,GO,BRASIL,SBNV,2021-06-16,15:50:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRKDT,<NA>,AVIÃO,PIPER AIRCRAFT,PA-28-161,P28A,PISTÃO,MONOMOTOR,1107.0,4.0,1992.0,BRASIL,AVIÃO,INSTRUÇÃO,AERÓDROMO NACIONAL DE AVIAÇÃO,AERÓDROMO NACIONAL DE AVIAÇÃO,DECOLAGEM,INSTRUÇÃO,LEVE,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
914,80119,ACIDENTE,RONDONÓPOLIS,MT,BRASIL,<NA>,2021-06-08,20:30:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTKCH,<NA>,AVIÃO,CESSNA AIRCRAFT,310Q,C310,PISTÃO,BIMOTOR,2404.0,6.0,1974.0,BRASIL,AVIÃO,PARTICULAR,AGROPASTORIL BOM PASTOR,AGROPASTORIL BOM PASTOR,CRUZEIRO,PRIVADA,DESTRUÍDA,4.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
942,80087,ACIDENTE,POCONÉ,MT,BRASIL,<NA>,2021-05-15,15:20:00,0,1,NÃO,NaN,NaN,NaN,NaN,PTJJH,<NA>,AVIÃO,CESSNA AIRCRAFT,182P,C182,PISTÃO,MONOMOTOR,1338.0,4.0,1974.0,BRASIL,AVIÃO,PARTICULAR,NÃO IDENTIFICADO,POCONÉ,CRUZEIRO,PRIVADA,SUBSTANCIAL,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
957,80078,INCIDENTE,GOIÂNIA,GO,BRASIL,<NA>,2021-05-08,19:00:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRZIG,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,SEM TRAÇÃO,0.0,0.0,0.0,BRASIL,<NA>,EXPERIMENTAL,NÃO IDENTIFICADO,NÃO IDENTIFICADO,DESCIDA DE EMERGÊNCIA,PRIVADA,NENHUM,0.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,NaN,NaN,NaN,NaN
1018,80005,ACIDENTE,TANGARÁ DA SERRA,MT,BRASIL,SWTS,2021-03-31,14:00:00,0,1,NÃO,NaN,NaN,NaN,NaN,PRZJW,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,SEM TRAÇÃO,0.0,0.0,0.0,BRASIL,<NA>,EXPERIMENTAL,TANGARÁ DA SERRA,CÁCERES,DECOLAGEM,PRIVADA,DESTRUÍDA,2.0,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCION

In [ ]:
dfac2021.groupby(['saida_pista'], dropna=False).size().sort_values(ascending=False)

saida_pista
NÃO    19
dtype: int64